In [ ]:
import os
root_dir = "/content/drive/My Drive"
from google.colab import drive
drive.mount('/content/drive')
project_folder = "/Colab Notebooks/chatbot-files"
os.chdir(root_dir + project_folder)

Mounted at /content/drive


In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)

intents

{'intents': [{'tag': 'google',
   'patterns': ['google', 'search', 'internet'],
   'responses': ['Redirecting to Google...']},
  {'tag': 'greeting',
   'patterns': ['Hi there',
    'How are you',
    'Is anyone there?',
    'Hey',
    'Hola',
    'Hello',
    'Good day',
    'Namaste',
    'yo'],
   'responses': ['Hello',
    'Good to see you again',
    'Hi there, how can I help?'],
   'context': ['']},
  {'tag': 'goodbye',
   'patterns': ['Bye',
    'See you later',
    'Goodbye',
    'Get lost',
    'Till next time',
    'bbye'],
   'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'],
   'context': ['']},
  {'tag': 'thanks',
   'patterns': ['Thanks',
    'Thank you',
    "That's helpful",
    'Awesome, thanks',
    'Thanks for helping me'],
   'responses': ['Happy to help!', 'Any time!', 'My pleasure'],
   'context': ['']},
  {'tag': 'noanswer',
   'patterns': [],
   'responses': ["Sorry, can't understand you",
    'Please give me more info',
    'Not sure I u

In [ ]:
import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('intents.json').read()
intents = json.loads(data_file)


# Access the 'intents' value and print the head
head_intents = intents['intents'][:3]  # Adjust the number of items as needed

for intent in head_intents:
    print(intent)

# Get the head of the intents dictionary
head_items = list(intents.items())[:5]  # Get the first 5 items

# Print the head items
for key, value in head_items:
    print(key, value)


#processing data

import nltk
nltk.download('punkt')
nltk.download('wordnet')

for intent in intents['intents']:
    for pattern in intent['patterns']:

        #tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        #add documents in the corpus
        documents.append((w, intent['tag']))

        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# lemmatize, lower each word and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
# sort classes
classes = sorted(list(set(classes)))
# documents = combination between patterns and intents
print (len(documents), "documents")
# classes = intents
print (len(classes), "classes", classes)
# words = all words, vocabulary
print (len(words), "unique lemmatized words", words)

pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))



#create testing and training data
# create our training data
training = []
# create an empty array for our output
output_empty = [0] * len(classes)
# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")


#build the model
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")



import nltk
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res) if r>ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

def getResponse(ints, intents_json):
    tag = ints[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return result

def chatbot_response(text):
    ints = predict_class(text, model)
    res = getResponse(ints, intents)
    return res

import ipywidgets as widgets
from IPython.display import display

def send(button):
    msg = entry_box.value.strip()
    entry_box.value = ''

    if msg != '':
        chat_log.value += f"You: {msg}\n\n"
        res = chatbot_response(msg)
        chat_log.value += f"Bot: {res}\n\n"
        chat_log.layout.display = 'none'
        chat_log.layout.display = 'flex'
        chat_log.value = chat_log.value

# Create Chat window
chat_log = widgets.Textarea(value='', layout=widgets.Layout(width='400px', height='400px'), disabled=True)

# Create Button to send message
send_button = widgets.Button(description='Send', layout=widgets.Layout(width='100px', height='50px'))
send_button.on_click(send)

# Create the box to enter message
entry_box = widgets.Textarea(value='', layout=widgets.Layout(width='300px', height='100px'))

# Display the components
display(chat_log, entry_box, send_button)


{'tag': 'google', 'patterns': ['google', 'search', 'internet'], 'responses': ['Redirecting to Google...']}
{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day', 'Namaste', 'yo'], 'responses': ['Hello', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}
{'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Get lost', 'Till next time', 'bbye'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}
intents [{'tag': 'google', 'patterns': ['google', 'search', 'internet'], 'responses': ['Redirecting to Google...']}, {'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day', 'Namaste', 'yo'], 'responses': ['Hello', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Get lost', 'Till next time', 'bbye'], 'responses':

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
<ipython-input-19-e706bc26e642>:95: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


Epoch 1/200
23/23 [==============================] - 1s 3ms/step - loss: 3.4007 - accuracy: 0.0442
Epoch 2/200
23/23 [==============================] - 0s 2ms/step - loss: 3.3722 - accuracy: 0.0354
Epoch 3/200
23/23 [==============================] - 0s 3ms/step - loss: 3.3120 - accuracy: 0.0885
Epoch 4/200
23/23 [==============================] - 0s 3ms/step - loss: 3.2309 - accuracy: 0.1150
Epoch 5/200
23/23 [==============================] - 0s 2ms/step - loss: 3.1334 - accuracy: 0.1858
Epoch 6/200
23/23 [==============================] - 0s 2ms/step - loss: 3.1198 - accuracy: 0.1327
Epoch 7/200
23/23 [==============================] - 0s 3ms/step - loss: 3.0646 - accuracy: 0.1858
Epoch 8/200
23/23 [==============================] - 0s 3ms/step - loss: 2.9561 - accuracy: 0.2301
Epoch 9/200
23/23 [==============================] - 0s 3ms/step - loss: 2.8789 - accuracy: 0.2566
Epoch 10/200
23/23 [==============================] - 0s 2ms/step - loss: 2.7924 - accuracy: 0.2389
Epoch 11/

Textarea(value='', disabled=True, layout=Layout(height='400px', width='400px'))

Textarea(value='', layout=Layout(height='100px', width='300px'))

Button(description='Send', layout=Layout(height='50px', width='100px'), style=ButtonStyle())

with BERT but no sparky

In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import json
import pickle
import numpy as np
import random
import nltk
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf

# Load intents from JSON file
data_file = open('intents.json').read()
intents = json.loads(data_file)

# Initialize tokenizer and BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

# Initialize lemmatizer for text preprocessing
lemmatizer = WordNetLemmatizer()

# Preprocess the data
words = []
classes = []
documents = []
ignore_words = ['?', '!']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Add documents to the corpus
        documents.append((w, intent['tag']))
        # Add to the classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize, lower each word, and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# Save words and classes to pickle files
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Load words and classes from pickle files
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])

random.shuffle(training)
training = np.array(training)
train_x = list(training[:, 0])
train_y = list(training[:, 1])

# Convert training data to tensors
train_x = tf.convert_to_tensor(train_x)
train_y = tf.convert_to_tensor(train_y)

# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_y[0]), activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Save the model
model.save('chatbot_model.h5')

# Load the model
model = tf.keras.models.load_model('chatbot_model.h5')

# Function to preprocess a sentence
def preprocess_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Function to convert a sentence to a BERT-compatible input
def convert_sentence_to_input(sentence):
    input_ids = []
    token_type_ids = []
    attention_mask = []
    encoded_input = tokenizer.encode_plus(sentence, add_special_tokens=True, return_tensors='tf')
    input_ids.append(encoded_input['input_ids'][0])
    token_type_ids.append(encoded_input['token_type_ids'][0])
    attention_mask.append(encoded_input['attention_mask'][0])
    input_ids = tf.convert_to_tensor(input_ids)
    token_type_ids = tf.convert_to_tensor(token_type_ids)
    attention_mask = tf.convert_to_tensor(attention_mask)
    return {'input_ids': input_ids, 'token_type_ids': token_type_ids, 'attention_mask': attention_mask}

# Function to predict the intent of a sentence
def predict_intent(sentence):
    input = convert_sentence_to_input(sentence)
    outputs = model(input)
    intent_index = tf.argmax(outputs, axis=1).numpy()[0]
    intent = classes[intent_index]
    return intent

# Function to get a random response for a given intent
def get_response(intent):
    for i in intents['intents']:
        if i['tag'] == intent:
            return random.choice(i['responses'])




Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.
<ipython-input-20-9226376c923a>:66: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or nda

Epoch 1/200
23/23 [==============================] - 1s 4ms/step - loss: 3.4029 - accuracy: 0.0177
Epoch 2/200
23/23 [==============================] - 0s 4ms/step - loss: 3.3728 - accuracy: 0.0708
Epoch 3/200
23/23 [==============================] - 0s 4ms/step - loss: 3.3278 - accuracy: 0.0708
Epoch 4/200
23/23 [==============================] - 0s 4ms/step - loss: 3.3092 - accuracy: 0.1062
Epoch 5/200
23/23 [==============================] - 0s 4ms/step - loss: 3.2471 - accuracy: 0.0973
Epoch 6/200
23/23 [==============================] - 0s 4ms/step - loss: 3.1973 - accuracy: 0.1416
Epoch 7/200
23/23 [==============================] - 0s 4ms/step - loss: 3.1551 - accuracy: 0.1416
Epoch 8/200
23/23 [==============================] - 0s 3ms/step - loss: 3.0899 - accuracy: 0.1416
Epoch 9/200
23/23 [==============================] - 0s 4ms/step - loss: 2.9949 - accuracy: 0.2212
Epoch 10/200
23/23 [==============================] - 0s 4ms/step - loss: 2.9533 - accuracy: 0.1947
Epoch 11/

In [ ]:
while True:
    msg = input("You: ")

    if msg.lower() == 'exit':
        break

    res = chatbot_response(msg)
    print("Bot:", res)


with BERT and more sparky

In [ ]:
from pyspark.ml.feature import Tokenizer
from pyspark.sql import SparkSession
from transformers import BertTokenizer, TFBertModel
import tensorflow as tf
import nltk
from nltk.stem import WordNetLemmatizer
import pickle
import random
import json
import numpy as np

# Initialize SparkSession
spark = SparkSession.builder.appName("Chatbot").getOrCreate()

# Load intents from JSON file
data_file = open('intents.json').read()
intents = json.loads(data_file)

# Initialize tokenizer and BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = TFBertModel.from_pretrained('bert-base-uncased')

# Initialize lemmatizer for text preprocessing
lemmatizer = WordNetLemmatizer()

# Preprocess the data
words = []
classes = []
documents = []
ignore_words = ['?', '!']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # Tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # Add documents to the corpus
        documents.append((w, intent['tag']))
        # Add to the classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

# Lemmatize, lower each word, and remove duplicates
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))

# Save words and classes to pickle files
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

# Load words and classes from pickle files
words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))

# Create training data
training = []
output_empty = [0] * len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append((bag, output_row))

random.shuffle(training)
training_rdd = spark.sparkContext.parallelize(training)
train_x = training_rdd.map(lambda x: x[0])
train_y = training_rdd.map(lambda x: x[1])

# Convert training data to tensors
train_x = train_x.collect()
train_y = train_y.collect()

train_x = tf.convert_to_tensor(train_x)
train_y = tf.convert_to_tensor(train_y)

# Build the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(128, input_shape=(len(train_x[0]),), activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(len(train_y[0]), activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(train_x, train_y, epochs=200, batch_size=5, verbose=1)

# Save the model
model.save('chatbot_model.h5')

# Load the model
model = tf.keras.models.load_model('chatbot_model.h5')

# Function to preprocess a sentence
def preprocess_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words

# Function to convert a sentence to a BERT-compatible input
def convert_sentence_to_input(sentence):
    input_ids = []
    token_type_ids = []
    attention_mask = []
    encoded_input = tokenizer.encode_plus(sentence, add_special_tokens=True, return_tensors='tf')
    input_ids.append(encoded_input['input_ids'][0])
    token_type_ids.append(encoded_input['token_type_ids'][0])
    attention_mask.append(encoded_input['attention_mask'][0])
    input_ids = tf.convert_to_tensor(input_ids)
    token_type_ids = tf.convert_to_tensor(token_type_ids)
    attention_mask = tf.convert_to_tensor(attention_mask)
    return {'input_ids': input_ids, 'token_type_ids': token_type_ids, 'attention_mask': attention_mask}

# Function to predict the intent of a sentence
def predict_intent(sentence):
    input = convert_sentence_to_input(sentence)
    outputs = model(input)
    intent_index = tf.argmax(outputs, axis=1).numpy()[0]
    intent = classes[intent_index]
    return intent

# Function to get a random response for a given intent
def get_response(intent):
    for i in intents['intents']:
        if i['tag'] == intent:
            return random.choice(i['responses'])


Some layers from the model checkpoint at bert-base-uncased were not used when initializing TFBertModel: ['nsp___cls', 'mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


Epoch 1/200
23/23 [==============================] - 2s 4ms/step - loss: 3.4132 - accuracy: 0.0442
Epoch 2/200
23/23 [==============================] - 0s 3ms/step - loss: 3.3711 - accuracy: 0.0885
Epoch 3/200
23/23 [==============================] - 0s 3ms/step - loss: 3.3618 - accuracy: 0.0531
Epoch 4/200
23/23 [==============================] - 0s 3ms/step - loss: 3.3250 - accuracy: 0.1062
Epoch 5/200
23/23 [==============================] - 0s 3ms/step - loss: 3.2584 - accuracy: 0.1416
Epoch 6/200
23/23 [==============================] - 0s 3ms/step - loss: 3.2189 - accuracy: 0.1947
Epoch 7/200
23/23 [==============================] - 0s 3ms/step - loss: 3.1687 - accuracy: 0.1239
Epoch 8/200
23/23 [==============================] - 0s 4ms/step - loss: 3.1088 - accuracy: 0.2301
Epoch 9/200
23/23 [==============================] - 0s 4ms/step - loss: 3.0259 - accuracy: 0.2124
Epoch 10/200
23/23 [==============================] - 0s 3ms/step - loss: 2.9929 - accuracy: 0.2301
Epoch 11/

In [ ]:
while True:
    msg = input("You: ")

    if msg.lower() == 'exit':
        break

    res = chatbot_response(msg)
    print("Bot:", res)
